In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib2
import json
import re

In [59]:
import string
def beep():
    print("\a")

In [60]:
df = pd.read_csv(r"C:\Users\SHIVAM MAHAJAN\Desktop\Users.csv")
query = "Farage OR Cameron OR Osborne OR Gove OR Johnson"      # Parameter for request to be made (Redundant)

In [61]:
class sentiment140:
    def __init__(self,df,char):
        self.data = []
        self.method = char
        self.df = df
        self.s = {}
        
    def jsonData(self,df):
        txt =  re.sub('#\S*|https\S*|@\S*','',df['tweet']) # Removing hashtags, URLs, @... from tweets
        hash = {}
        txt = re.sub(r"\\x[a-z0-9][a-z0-9]",'',repr(txt))  # For dealing with \x0f\xa2 etc - smilies/emotions
        txt = re.sub(r'\\n','',txt)                        # converting back - repr(text) -> text
        txt = re.sub(r"'|,",'',txt)                        # Substituting ' and , with "" (Nothing)
        txt = txt.strip()                                  # Removing extra spacing around the string
        hash["text"] = str(txt)                              
        hash["query"] = query
        self.data.append(hash)            
        
    def jsonFormat(self,init,end):                 
        df2 = self.df.iloc[init:end,:]             # Sentiment140 can analyze sentiment of only 10000 tweets at a time
        df2.apply(self.jsonData,axis=1).head()     #  Applying jsonData() method to every row of data frame.
        self.s["data"] = self.data                 # Foring JSON data
         
    def recieveSentiment(self,init,end):
        self.jsonFormat(init,end)
        response = urllib2.urlopen('http://www.sentiment140.com/api/bulkClassifyJson', str(self.s)) # request to server
        page = response.read()                                                                      # get the response
        js=json.loads(page)                  # Getting JSON
        size = len(js["data"])               
        f = open("C:\Users\SHIVAM MAHAJAN\Desktop\sentiment2.csv",self.method)
        
        for i in range(size):                # Storing in file - sentiment2.csv both tweet and sentiment
            sent=""
            txt = str(js["data"][i]["text"])
            txt = string.replace(txt,'\n',' ')
            sent += str(txt) + "," + str(js["data"][i]["polarity"]) + "\n"
            f.write(sent)
            
        f.close()

In [62]:
senti = sentiment140(df,"w")
senti.recieveSentiment(0,100)
if True:
    beep()




In [63]:
df3 = pd.read_csv(r"C:\Users\SHIVAM MAHAJAN\Desktop\sentiment2.csv",header=None,names=["tweets","Sentiment"],sep=',')